In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.pandaz.utils import *

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers import PortfolioRebalancerTracker
from qubx.core.series import Quote
from qubx.data.readers import CsvStorageDataReader, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC
from qubx.core.basics import Deal, Instrument, Order, Position, Signal
from qubx.core.utils import recognize_time

from qubx.impl.ccxt_connector import CCXTDataConnector
from qubx.impl.ccxt_trading import CCXTSyncTradingConnector
from qubx.utils.runner import get_account_config

def Q(time: str, bid: float, ask: float) -> Quote:
    return Quote(recognize_time(time), bid, ask, 0, 0)

 >  [dev] installing cython rebuilding hook

⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        
2024-07-10 16:48:39.149 [ ℹ️ ] NTP offset controller thread is started


# Simulated broker and paper trading

In [23]:
from qubx.backtester.simulator import SimulatedExchangeService

In [24]:
class TestStrategy(IStrategy):
    capital_invested: int = 1000

    def on_start(self, ctx: StrategyContext):
        logger.info(f"Strategy started | {ctx.time()}")

    def on_fit(self, ctx: 'StrategyContext', fit_time: str | pd.Timestamp, previous_fit_time: str | pd.Timestamp | None = None):
        logger.info(f"> Fit is called | fit_time: {fit_time} / prev: {previous_fit_time}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        logger.info(f"{event.time} -> {event}")
        return None

    def ohlcs(self, timeframe:str) -> Dict[str, pd.DataFrame]:
        return {s.symbol: self.ctx.ohlc(s, timeframe).pd() for s in self.ctx.instruments}

    def on_stop(self, ctx: StrategyContext):
        logger.info(f"> test is stopped")

    def tracker(self, ctx: StrategyContext) -> PortfolioRebalancerTracker:
        return PortfolioRebalancerTracker(ctx, 1000, 0)

In [25]:
auth = get_account_config('binance-qbx', './configs/.env')

sbroker = SimulatedExchangeService('binance', 10000, 'vip0_usdt', 'USDT')
data = CCXTDataConnector('binance', sbroker, **auth)

instruments = [lookup.find_symbol('BINANCE', 'BTCUSDT')]
ctx = StrategyContext(
    TestStrategy(), dict(
        capital_invested=100,
        trading_allowed=False
    ), 
    data, sbroker, instruments=instruments,  # type: ignore
    md_subscription=dict(type='ohlc', timeframe='1Min', nback=0),
    trigger_spec='1Min -5Sec',
)
S = ctx.strategy

2024-07-10 17:00:49.402 [ ℹ️ ] binance initialized - current time 1970-01-01T00:00:00.000000000
2024-07-10 17:00:49.403 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-10 17:00:49.404 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-10 17:00:49.405 [ 🐞 ] Triggering strategy on every 1Min bar after 55S
2024-07-10 17:00:49.403 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-10 17:00:49.404 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-10 17:00:49.405 [ 🐞 ] Triggering strategy on every 1Min bar after 55S


In [ ]:
ctx.start()

2024-07-10 17:00:52.328 [ ℹ️ ] (StrategyContext) Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 0} for 
	['BTCUSDT'] 
2024-07-10 17:00:52.328 [ ℹ️ ] Subscribed on ohlc updates for 1 symbols: 
		['BTCUSDT']
2024-07-10 17:00:52.330 [ ℹ️ ] Strategy started | 1970-01-01T00:00:00.000000000
2024-07-10 17:00:52.332 [ ℹ️ ] (StrategyContext) Start processing market data
2024-07-10 17:00:52.333 [ ℹ️ ] (StrategyContext) strategy is started


2024-07-10 17:00:59.141 [ ⚠️ ] qubx.core.strategy:_process_incoming_data:452 - [2024-07-10T14:00:59.576910848] TestStrategy::on_event() is SKIPPED for now because on_fit() was not called yet !
2024-07-10 17:00:59.141 [ 🐞 ] [2024-07-10T14:00:59.576910848]: Invoking TestStrategy on_fit('1970-01-01 00:00:00', 'None')
2024-07-10 17:00:59.141 [ ℹ️ ] > Fit is called | fit_time: 1970-01-01 00:00:00 / prev: None
2024-07-10 17:00:59.141 [ 🐞 ] [2024-07-10T14:00:59.576910848]: TestStrategy is fitted
2024-07-10 17:01:15.264 [ 🐞 ] [SIM-ORDER-BTCUSDT-100001 / TestStrategy_BTCUSDT_1] : LIMIT BUY 0.1 of BTCUSDT  @ 45000 -> [OPEN]
2024-07-10 17:01:27.179 [ 🐞 ] [SIM-ORDER-BTCUSDT-100002 / TestStrategy_BTCUSDT_2] : LIMIT SELL 0.1 of BTCUSDT  @ 45000 -> [CLOSED]
2024-07-10 17:01:27.179 [ 🐞 ] Executed -0.1 @ 57780.495 of BTCUSDT for order SIM-ORDER-BTCUSDT-100002
2024-07-10 17:01:39.318 [ 🐞 ] [SIM-ORDER-BTCUSDT-100003 / TestStrategy_BTCUSDT_3] : MARKET BUY 0.1 of BTCUSDT  -> [CLOSED]
2024-07-10 17:01:39.31

In [42]:
str(sbroker.get_position(instruments[0]))

'2024-07-10 14:01:58 [BINANCE:BTCUSDT]       0.00000      0.00    -4.5860 | 57789.33        0.00'

In [39]:
ctx.trade('BTCUSDT', 0.1)

2024-07-10 17:01:44.892 [ ℹ️ ] (StrategyContext) sending market buy for 0.1 of BTCUSDT ...
2024-07-10 17:01:44.892 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100004 MARKET BUY 0.1 executed at 57798.094999999994
2024-07-10 17:01:44.892 [ ℹ️ ]   ::  traded 0.1 for BTCUSDT @ 57798.094999999994 -> 0.00
2024-07-10 17:01:44.892 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100004 MARKET BUY 0.1 of BTCUSDT -> CLOSED


Order(id='SIM-ORDER-BTCUSDT-100004', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2024-07-10T14:01:43.401421056'), quantity=0.1, price=0, side='BUY', status='CLOSED', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_4', cost=0.0)

In [41]:
ctx.trade('BTCUSDT', -0.1)

2024-07-10 17:01:55.173 [ ℹ️ ] (StrategyContext) sending market sell for 0.1 of BTCUSDT ...
2024-07-10 17:01:55.174 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100005 MARKET SELL 0.1 executed at 57784.145000000004
2024-07-10 17:01:55.175 [ ℹ️ ]   ::  traded -0.1 for BTCUSDT @ 57784.145000000004 -> -1.39
2024-07-10 17:01:55.176 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100005 MARKET SELL 0.1 of BTCUSDT -> CLOSED


Order(id='SIM-ORDER-BTCUSDT-100005', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2024-07-10T14:01:54.048028160'), quantity=0.1, price=0, side='SELL', status='CLOSED', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_5', cost=0.0)

In [43]:
ctx.stop()
data.close()

2024-07-10 17:02:05.861 [ ℹ️ ] > test is stopped
2024-07-10 17:02:05.861 [ ℹ️ ] 	CachedMarketDataHolder.update_by_bar took 0.0000215 secs
2024-07-10 17:02:05.861 [ ℹ️ ] 	StrategyContext._processing_bar took 0.0000314 secs
2024-07-10 17:02:05.861 [ ℹ️ ] 	StrategyContext.handler took 0.0000024 secs
2024-07-10 17:02:05.861 [ ℹ️ ] 	StrategyContext._process_incoming_data took 1.4704930 secs
2024-07-10 17:02:05.861 [ ℹ️ ] 	loggers.notify took 0.0000035 secs
2024-07-10 17:02:05.861 [ ℹ️ ] 	strategy.on_fit took 0.0012412 secs
2024-07-10 17:02:05.873 [ ℹ️ ] 	strategy.on_event took 0.0013669 secs
2024-07-10 17:02:05.873 [ ℹ️ ] 	StrategyContext.trade took 0.0025071 secs
2024-07-10 17:02:05.873 [ ℹ️ ] 	StrategyContext._processing_order took 0.0008409 secs
2024-07-10 17:02:05.874 [ ℹ️ ] 	StrategyContext._processing_deals took 0.0012512 secs
2024-07-10 17:02:05.874 [ ❌ ] This event loop is already running
2024-07-10 17:02:05.861 [ ℹ️ ] 	CachedMarketDataHolder.update_by_bar took 0.0000215 secs
2024-0

# Simulated data provider

In [ ]:
r = CsvStorageDataReader('../tests/data/csv')

In [9]:
instr = lookup.find_symbol("BINANCE.UM", "BTCUSDT")
stream = r.read('BTCUSDT_ohlcv_M1', transform=RestoreTicksFromOHLC(trades=False, spread=instr.min_tick))